## 1. Configuration et Imports

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuration graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuration API
API_BASE_URL = "http://localhost:8000"  # Modifier si API distante

## 2. Connexion à l'API

In [ ]:
# Test de connexion
response = requests.get(f"{API_BASE_URL}/")
print(f"Status API: {response.json()}")

# Statistiques globales
stats = requests.get(f"{API_BASE_URL}/statistics").json()
print(f"\nStatistiques globales:")
print(f"  - Total accidents: {stats.get('total_accidents', 'N/A')}")
print(f"  - Période: {stats.get('date_range', 'N/A')}")

## 3. Chargement des Données

In [ ]:
def load_accidents_data(limit=1000, offset=0):
    """Charge les données d'accidents via API"""
    url = f"{API_BASE_URL}/accidents?limit={limit}&offset={offset}"
    response = requests.get(url)
    if response.status_code == 200:
        return pd.DataFrame(response.json())
    else:
        print(f"Erreur: {response.status_code}")
        return None

# Charger les données
df = load_accidents_data(limit=5000)
print(f"Données chargées: {len(df)} lignes, {len(df.columns)} colonnes")
df.head()

## 4. Analyse Descriptive

In [ ]:
# Informations générales
print("=" * 50)
print("INFORMATIONS SUR LE DATASET")
print("=" * 50)
df.info()

print("\n" + "=" * 50)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 50)
df.describe()

## 5. Valeurs Manquantes

In [ ]:
# Analyse des valeurs manquantes
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Valeurs Manquantes': missing,
    'Pourcentage': missing_pct
}).sort_values('Pourcentage', ascending=False)

print(missing_df[missing_df['Pourcentage'] > 0])

# Visualisation
plt.figure(figsize=(12, 6))
missing_data = missing_df[missing_df['Pourcentage'] > 0]
if not missing_data.empty:
    plt.barh(missing_data.index, missing_data['Pourcentage'])
    plt.xlabel('Pourcentage de valeurs manquantes')
    plt.title('Valeurs Manquantes par Colonne')
    plt.tight_layout()
    plt.show()
else:
    print("\n✅ Aucune valeur manquante détectée")

## 6. Distributions des Variables Clés

In [ ]:
# Distribution de la gravité
if 'grav' in df.columns:
    plt.figure(figsize=(10, 5))
    df['grav'].value_counts().sort_index().plot(kind='bar')
    plt.title('Distribution de la Gravité des Accidents')
    plt.xlabel('Gravité')
    plt.ylabel('Nombre d\'accidents')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

In [ ]:
# Distribution temporelle
if 'an' in df.columns:
    plt.figure(figsize=(12, 5))
    df['an'].value_counts().sort_index().plot(kind='line', marker='o')
    plt.title('Évolution du Nombre d\'Accidents par Année')
    plt.xlabel('Année')
    plt.ylabel('Nombre d\'accidents')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 7. Corrélations

In [ ]:
# Matrice de corrélation pour les variables numériques
numeric_cols = df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 1:
    corr_matrix = df[numeric_cols].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('Matrice de Corrélation')
    plt.tight_layout()
    plt.show()
else:
    print("Pas assez de variables numériques pour calculer les corrélations")

## 8. Résumé de l'Exploration

In [ ]:
print("=" * 60)
print("RÉSUMÉ DE L'EXPLORATION")
print("=" * 60)
print(f"Nombre de lignes: {len(df)}")
print(f"Nombre de colonnes: {len(df.columns)}")
print(f"Mémoire utilisée: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nTypes de données:")
print(df.dtypes.value_counts())
print("\n✅ Exploration terminée!")

## 📝 Notes

**Prochaines étapes** :
1. Passer au notebook `02_statistical_analysis.ipynb` pour analyses statistiques
2. Identifier les features importantes pour le ML
3. Nettoyer les données si nécessaire

**Documentation API** : Voir `/docs` sur l'API REST